In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas
import numpy 

Lendo arquivo e retirando coluna 'english', 'median_playtime', 'required_age'

In [0]:
df = pandas.read_csv('/content/drive/My Drive/steam.csv').drop(columns=['english', 'median_playtime', 'required_age'])

Calculando porcentagem de analises positivas

In [0]:
df['rate'] = df['positive_ratings'] / (df['positive_ratings'] + df['negative_ratings'])

Dividindo dados de acordo com rate para facilitar a classificação

In [0]:
df1 = df[df['rate'] <= 0.50]
df2 = df[df['rate'] > 0.50]

Aplicando classificação para cada faixa de rate

In [0]:
df1['c'] = 'negativo'
df2['c'] = 'positivo'

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Concatenando os dados que foram divididos

In [0]:
df = pandas.concat([df1, df2])

Retirando atributos que nao serão mais utilizados

In [0]:
df = df.drop(columns=['positive_ratings', 'negative_ratings', 'rate'])

In [0]:
df['average_playtime'].fillna(df['average_playtime'].mean(), inplace=True)

In [0]:
df = df.sort_values('appid')
df = df.drop(columns=['appid'])

In [0]:
df = df.replace('0-20000', '10000')
df = df.replace('20000-50000', '35000.0')
df = df.replace('50000-100000', '75000.0')
df = df.replace('100000-200000', '150000.0')
df = df.replace('200000-500000', '350000.0')
df = df.replace('500000-1000000', '750000.0')
df = df.replace('1000000-2000000', '1500000.0')
df = df.replace('2000000-5000000', '3500000.0')
df = df.replace('5000000-10000000', '7500000.0')
df = df.replace('10000000-20000000', '15000000.0')
df = df.replace('20000000-50000000', '35000000.0')
df = df.replace('50000000-100000000', '75000000.0')
df = df.replace('100000000-200000000', '150000000')

Dummy dos genres

In [0]:
genres = (df['genres'].apply(lambda x: str(x)
                                     .split(",")))
genres = genres.to_list()
unique_list_of_genres = []
for list_of_each_game in genres:
    splited_string = list_of_each_game[0].split(';')
    for genre in splited_string:
        unique_list_of_genres.append(genre)

unique_list_of_genres = list(dict.fromkeys(unique_list_of_genres))
for genre in unique_list_of_genres:
    df[genre] = df['genres'].apply(lambda x: 1 if genre in x.split(';') else 0)

Dummy das categories

In [0]:
categories = (df['categories'].apply(lambda x: str(x)
                                     .split(",")))
categories = categories.to_list()
unique_list_of_categories = []
for list_of_each_game in categories:
    splited_string = list_of_each_game[0].split(';')
    for category in splited_string:
        unique_list_of_categories.append(category)

unique_list_of_categories = list(dict.fromkeys(unique_list_of_categories))
for category in unique_list_of_categories:
    df['C_'+category] = df['categories'].apply(lambda x: 1 if category in x.split(';') else 0)

In [0]:
df.columns

In [0]:
print(df.loc[1])

Começo da etapa de separação de dados para a classificação
__________________________________________________________

Separação de atributos para classificação, e a classificação em si (X e y).

In [0]:
X = df.iloc[:, [10, 11, 13, 14, 15, 16, 17, 18, 19, 20 ,21 ,22, 23, 24, 25, 26, 26, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48
, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70]].values
y = df.iloc[:, 12].values

print(X.shape)
print(y.shape)

print(df.columns)

(27075, 60)
(27075,)
Index(['name', 'release_date', 'developer', 'publisher', 'platforms',
       'categories', 'genres', 'steamspy_tags', 'achievements',
       'average_playtime', 'owners', 'price', 'c', 'Action', 'Free to Play',
       'Strategy', 'Adventure', 'Indie', 'RPG', 'Animation & Modeling',
       'Video Production', 'Casual', 'Simulation', 'Racing', 'Violent',
       'Massively Multiplayer', 'Nudity', 'Sports', 'Early Access', 'Gore',
       'Utilities', 'Design & Illustration', 'Web Publishing', 'Education',
       'Software Training', 'Sexual Content', 'Audio Production',
       'Game Development', 'Photo Editing', 'Accounting', 'Documentary',
       'Tutorial', 'C_Multi-player', 'C_Online Multi-Player',
       'C_Local Multi-Player', 'C_Valve Anti-Cheat enabled', 'C_Single-player',
       'C_Steam Cloud', 'C_Steam Achievements', 'C_Steam Trading Cards',
       'C_Captions available', 'C_Partial Controller Support',
       'C_Includes Source SDK', 'C_Cross-Platform Multi

Utilização da função SelectKBest para selecionar os 10 melhores atributos para a classificação

In [0]:
from sklearn.feature_selection import SelectKBest, chi2

X = SelectKBest(chi2, k=10).fit_transform(X, y)


Separação dos dados de treino e dados de predição
---



In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)


Classificando com arvore de decisão
---



In [0]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics


clf = DecisionTreeClassifier()


In [0]:
accuracy_dt_sum = 0
for i in range(0, 10):
  clf = clf.fit(X_train,y_train)
  y_pred = clf.predict(X_test)
  accuracy_dt_sum += metrics.accuracy_score(y_test, y_pred)

accuracy_dt_avg = accuracy_dt_sum/10

Calculando média da acurácia

In [0]:
print(accuracy_dt_avg)

Classificando com KNN (11 vizinhos)
---

In [0]:
from sklearn.neighbors import KNeighborsClassifier

classifier_knn11 = KNeighborsClassifier(n_neighbors=11)

In [0]:
accuracy_knn11_sum = 0
for i in range(0, 10):
  classifier_knn11.fit(X_train, y_train)
  y_pred = classifier_knn11.predict(X_test)
  accuracy_knn11_sum += metrics.accuracy_score(y_test, y_pred)

accuracy_knn11_avg = accuracy_knn11_sum/10

Calculando média da acurácia

In [0]:
print(accuracy_knn11_avg)

Classificando com KNN (7 vizinhos)
---

In [0]:
from sklearn.neighbors import KNeighborsClassifier

classifier_knn7 = KNeighborsClassifier(n_neighbors=7)

In [0]:
accuracy_knn7_sum = 0
for i in range(0, 10):
  classifier_knn7.fit(X_train, y_train)
  y_pred = classifier_knn7.predict(X_test)
  accuracy_knn7_sum += metrics.accuracy_score(y_test, y_pred)

accuracy_knn7_avg = accuracy_knn7_sum/10

Calculando média da acurácia

In [0]:
print(accuracy_knn7_avg)

Classificando com KNN (5 vizinhos)
---

In [0]:
from sklearn.neighbors import KNeighborsClassifier

classifier_knn5 = KNeighborsClassifier(n_neighbors=5)

In [0]:
accuracy_knn5_sum = 0
for i in range(0, 10):
  classifier_knn5.fit(X_train, y_train)
  y_pred = classifier_knn5.predict(X_test)
  accuracy_knn5_sum += metrics.accuracy_score(y_test, y_pred)

accuracy_knn5_avg = accuracy_knn5_sum/10

Calculando média da acurácia

In [0]:
print(accuracy_knn5_avg)

Classificando com KNN (3 vizinhos)
---

In [0]:
classifier_knn3 = KNeighborsClassifier(n_neighbors=3)

In [0]:
accuracy_knn3_sum = 0
for i in range(0, 10):
  classifier_knn3.fit(X_train, y_train)
  y_pred = classifier_knn3.predict(X_test)
  accuracy_knn3_sum += metrics.accuracy_score(y_test, y_pred)

accuracy_knn3_avg = accuracy_knn3_sum/10

Calculando média da acurácia

In [0]:
print(accuracy_knn3_avg)

**Refazendo testes sem utilizar SelectKBest**
---



In [0]:
X = df.iloc[:, [10, 11, 13, 14, 15, 16, 17, 18, 19, 20 ,21 ,22, 23, 24, 25, 26, 26, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48
, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70]].values
y = df.iloc[:, 12].values

print(X.shape)
print(y.shape)

(27075, 60)
(27075,)



Separação dos dados de treino e dados de predição
---



In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(18952, 60)
(18952,)
(8123, 60)
(8123,)



Classificando com arvore de decisão
---



In [0]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics


clf_no_select = DecisionTreeClassifier()


In [0]:
accuracy_dt_sum_no_select = 0
for i in range(0, 10):
  clf = clf_no_select.fit(X_train,y_train)
  y_pred = clf_no_select.predict(X_test)
  accuracy_dt_sum_no_select += metrics.accuracy_score(y_test, y_pred)

accuracy_dt_avg_no_select = accuracy_dt_sum_no_select/10

Calculando média da acurácia

In [0]:
print(accuracy_dt_avg_no_select)

Classificando com KNN (11 vizinhos)
---

In [0]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix

classifier_knn11_no_select = KNeighborsClassifier(n_neighbors=11)

In [0]:
accuracy_knn11_sum_no_select = 0
for i in range(0, 10):
  classifier_knn11_no_select.fit(X_train, y_train)
  y_pred = classifier_knn11_no_select.predict(X_test)
  accuracy_knn11_sum_no_select += metrics.accuracy_score(y_test, y_pred)
  print(metrics.accuracy_score(y_test, y_pred))
accuracy_knn11_avg_no_select = accuracy_knn11_sum_no_select/10

0.8027822233165087
0.8027822233165087
0.8027822233165087
0.8027822233165087
0.8027822233165087
0.8027822233165087
0.8027822233165087
0.8027822233165087
0.8027822233165087
0.8027822233165087


Calculando média da acurácia

In [0]:
print(metrics.confusion_matrix(y_test, y_pred))
print(accuracy_knn11_avg_no_select)

[[  42 1494]
 [ 241 6346]]
0.8027822233165087


Classificando com KNN (7 vizinhos)
---

In [0]:
from sklearn.neighbors import KNeighborsClassifier

classifier_knn7_no_select = KNeighborsClassifier(n_neighbors=7)

In [0]:
accuracy_knn7_sum_no_select = 0
for i in range(0, 10):
  classifier_knn7_no_select.fit(X_train, y_train)
  y_pred = classifier_knn7_no_select.predict(X_test)
  accuracy_knn7_sum_no_select += metrics.accuracy_score(y_test, y_pred)

accuracy_knn7_avg_no_select = accuracy_knn7_sum_no_select/10

Calculando média da acurácia

In [0]:
print(accuracy_knn7_avg_no_select)

Classificando com KNN (5 vizinhos)
---

In [0]:
from sklearn.neighbors import KNeighborsClassifier

classifier_knn5_no_select = KNeighborsClassifier(n_neighbors=5)

In [0]:
accuracy_knn5_sum_no_select = 0
for i in range(0, 10):
  classifier_knn5_no_select.fit(X_train, y_train)
  y_pred = classifier_knn5_no_select.predict(X_test)
  accuracy_knn5_sum_no_select += metrics.accuracy_score(y_test, y_pred)

accuracy_knn5_avg_no_select = accuracy_knn5_sum_no_select/10

Calculando média da acurácia

In [0]:
print(accuracy_knn5_avg_no_select)

Classificando com KNN (3 vizinhos)
---

In [0]:
classifier_knn3_no_select = KNeighborsClassifier(n_neighbors=3)

In [0]:
accuracy_knn3_sum_no_select = 0
for i in range(0, 10):
  classifier_knn3_no_select.fit(X_train, y_train)
  y_pred = classifier_knn3_no_select.predict(X_test)
  accuracy_knn3_sum_no_select += metrics.accuracy_score(y_test, y_pred)

accuracy_knn3_avg_no_select = accuracy_knn3_sum_no_select/10

Calculando média da acurácia

In [0]:
print(accuracy_knn3_avg_no_select)

In [0]:
import matplotlib.pyplot as plt
plt.figure(figsize=(15, 10))
plt.plot([accuracy_dt_avg, accuracy_knn11_avg, accuracy_knn7_avg, accuracy_knn5_avg, accuracy_knn3_avg, accuracy_dt_avg_no_select, accuracy_knn11_avg_no_select, accuracy_knn7_avg_no_select, accuracy_knn5_avg_no_select,accuracy_knn3_avg_no_select])

plt.text(0, accuracy_dt_avg, accuracy_dt_avg)
plt.text(1, accuracy_knn11_avg, accuracy_knn11_avg)
plt.text(2, accuracy_knn7_avg, accuracy_knn7_avg)
plt.text(3, accuracy_knn5_avg, accuracy_knn5_avg)
plt.text(4, accuracy_knn3_avg, accuracy_knn3_avg)
plt.text(5, accuracy_dt_avg_no_select, accuracy_dt_avg_no_select)
plt.text(6, accuracy_knn11_avg_no_select, accuracy_knn11_avg_no_select)
plt.text(7, accuracy_knn7_avg_no_select, accuracy_knn7_avg_no_select)
plt.text(8, accuracy_knn5_avg_no_select, accuracy_knn5_avg_no_select)
plt.text(9, accuracy_knn3_avg_no_select, accuracy_knn3_avg_no_select)

plt.xticks([i for i in range(0, 12)])
plt.yticks([0.73, 0.74, 0.75, 0.76, 0.77, 0.78, 0.79, 0.80, 0.81])
